In [12]:
from datetime import datetime
import functools
from IPython.display import HTML
import jax
from jax import numpy as jp
import numpy as np
from typing import Any, Dict, Sequence, Tuple, Union
from brax import base
from brax import envs
from brax import math
from brax.base import Base, Motion, Transform
from brax.envs.base import Env, PipelineEnv, State
from brax.mjx.base import State as MjxState
# from brax.training.agents.apg import train as apg
# from brax.training.agents.apg import networks as apg_networks
from brax.io import html, mjcf, model
from etils import epath
from flax import struct
from matplotlib import pyplot as plt
import mediapy as media
from ml_collections import config_dict
import mujoco
from mujoco import mjx
from jax import vmap
import jax.random
from jax import lax

In [13]:
import distutils.util
import os
xla_flags = os.environ.get('XLA_FLAGS', '')
xla_flags += ' --xla_gpu_triton_gemm_any=True'
os.environ['XLA_FLAGS'] = xla_flags
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

Setting environment variable to use GPU rendering:
env: MUJOCO_GL=egl


In [14]:
import os
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.8" # 0.9 causes too much lag. 
from datetime import datetime
import functools

# Math
import jax.numpy as jp
import numpy as np
import jax
from jax import config # Analytical gradients work much better with double precision.
config.update("jax_debug_nans", True)
config.update("jax_enable_x64", True)
config.update('jax_default_matmul_precision', jax.lax.Precision.HIGH)
from brax import math

# Sim
import mujoco
import mujoco.mjx as mjx

# Brax
from brax import envs
from brax.base import Motion, Transform
from brax.io import mjcf
from brax.envs.base import PipelineEnv, State
from brax.mjx.pipeline import _reformat_contact
from brax.training.acme import running_statistics
from brax.io import model

# Algorithms
# from brax.training.agents.apg import train as apg
# from brax.training.agents.apg import networks as apg_networks
from brax.training.agents.ppo import train as ppo

# Supporting
from etils import epath
import mediapy as media
import matplotlib.pyplot as plt
from ml_collections import config_dict
from typing import Any, Dict



In [15]:
from utils.SimpleConverter import SimpleConverter
from agent_mimic_env.agent_template import HumanoidTemplate
from agent_mimic_env.agent_eval_template import HumanoidEvalTemplate
from agent_mimic_env.agent_training_template import HumanoidTrainTemplate
from agent_mimic_env.agent_test_apg import HumanoidAPGTest
from agent_mimic_env.agent_eval_ppo import HumanoidPPOEval
from utils.util_data import *
from copy import deepcopy

In [16]:
import agent_mimic_env
from agent_mimic_env import register_mimic_env

In [17]:
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)

gpu


In [18]:
import yaml
from box import Box
# Path to your YAML file
yaml_file_path = 'config_params/punch.yaml'
# Load the YAML file
with open(yaml_file_path, 'r') as file:
    args = Box(yaml.safe_load(file))

In [19]:
from agent_mimic_env.pds_controllers_agents import feedback_pd_controller, stable_pd_controller_action

In [20]:

# env_replay,env_eval, env,env_apg,env_eval_ppo=register_mimic_env(args)

# #for the eval here we run the trained policy
# # jit_reset = jax.jit(env_eval.reset)
# # jit_step = jax.jit(env_eval.step)

# env_eval_ppo.set_pd_callback(stable_pd_controller_action)

# env_eval.set_pd_callback(stable_pd_controller_action)

# env.set_pd_callback(stable_pd_controller_action)
#env_apg.set_pd_callback(stable_pd_controller_action)

In [21]:
trajectory = SimpleConverter(args.ref)
trajectory.load_mocap()
model_path = 'models/final_humanoid.xml'
envs.register_environment('humanoidPPOEval',HumanoidPPOEval)
env_name_ppo_eval = 'humanoidPPOEval'
env_kwargs = dict(referece_data=trajectory,model_path=model_path,args=args) 
    
env_eval_ppo = envs.get_environment(env_name_ppo_eval,**env_kwargs)
    


In [22]:
episode_len = env_eval_ppo.rollout_lenght
print(episode_len)
#print(env.err_threshold)

65


In [23]:

from brax.training.agents.ppo import train as ppo
from brax.training.agents.ppo import networks as ppo_networks


In [24]:

make_networks_factory = functools.partial(
    ppo_networks.make_ppo_networks,
        policy_hidden_layer_sizes=(512,256),
        value_hidden_layer_sizes=(512,256))
make_networks_factory

functools.partial(<function make_ppo_networks at 0x7fa40b6dede0>, policy_hidden_layer_sizes=(512, 256), value_hidden_layer_sizes=(512, 256))

In [25]:
train_fn = functools.partial(
    ppo.train, num_timesteps=1000, num_evals=10,
    episode_length=episode_len-1, normalize_observations=True, action_repeat=1,
    unroll_length=15, num_minibatches=24, num_updates_per_batch=8,
    discounting=0.97, learning_rate=3e-4, entropy_cost=1e-3, num_envs=256,
    batch_size=256, seed=0,network_factory=make_networks_factory)


In [26]:
x_data = []
y_data = []
ydataerr = []
y_pose_error = []  # to store pose error
times = [datetime.now()]
max_y_rewards, min_y_rewards = 0, -5
def progress(num_steps, metrics):
  
  #print(num_steps)
  times.append(datetime.now())
  x_data.append(num_steps)
  y_data.append(metrics['eval/episode_reward'])
  ydataerr.append(metrics['eval/episode_reward_std'])
  #y_pose_error.append(metrics['eval/episode_pose_error'])  # capture pose error
  
  plt.figure(figsize=(10, 5))
  plt.subplot(1, 2, 1) 
  plt.xlim([0, num_steps* 1.25])
  plt.ylim([max_y_rewards, min_y_rewards])

  plt.xlabel('# environment steps')
  plt.ylabel('reward per episode')
  plt.title(f'Latest Reward: {y_data[-1]:.3f}')
  plt.plot(x_data, y_data, '-o')
  
  # plt.subplot(1, 2, 2)
  # plt.xlim([0, num_steps* 1.25])
  # plt.ylim([min(y_pose_error) * 0.9, max(y_pose_error) * 1.1])
  # plt.xlabel('# Environment Steps')
  # plt.ylabel('Pose Error per Episode')
  # plt.title(f'Latest Pose Error: {y_pose_error[-1]:.3f}')
  # plt.plot(x_data, y_pose_error, '-o')
  

  plt.tight_layout()
  plt.show()
  

make_inference_fn, params, _= train_fn(environment=env_eval_ppo,
                                       progress_fn=progress,
                                       eval_env=env_eval_ppo)

print(f'time to jit: {times[1] - times[0]}')
print(f'time to train: {times[-1] - times[1]}')

ConcretizationTypeError: Abstract tracer value encountered where concrete value is expected: traced array with shape int64[].
The problem arose with the `int` function. If trying to convert the data type of a value, try using `x.astype(int)` or `jnp.array(x, int)` instead.
The error occurred while tracing the function generate_eval_unroll at /home/user/anaconda3/envs/newBrax/lib/python3.11/site-packages/brax/training/acting.py:103 for jit. This value became a tracer due to JAX operations on these lines:

  operation a:i64[] = convert_element_type[new_dtype=int64 weak_type=False] b
    from line /home/user/workspace/RLPhysics/Physics-Based-RL/agent_mimic_env/agent_eval_ppo.py:155:15 (HumanoidPPOEval.reset)

  operation a:i64[] = convert_element_type[new_dtype=int64 weak_type=False] b
    from line /home/user/workspace/RLPhysics/Physics-Based-RL/agent_mimic_env/agent_eval_ppo.py:155:15 (HumanoidPPOEval.reset)

  operation a:i64[] = convert_element_type[new_dtype=int64 weak_type=False] b
